### NOTE: This is maintained for example purpose.  'Real' query process has been moved to 'cancer_data_query.py'

In [ ]:
import os
import re
import pathlib
import requests
import pandas as pd
pd.options.display.max_columns=999

# from tqdm import tqdm, tqdm_notebook

from functools import partial

from cancer_data_query import CancerDataQuery, query_state_data

In [2]:
# Define the data paths
project_dir = pathlib.Path.cwd().parent.parent
repo_dir = pathlib.Path.cwd().parent
raw_data_dir = repo_dir / 'data_raw'

In [3]:
raw_data_dir

PosixPath('/Users/merrelbook/Projects/HealthDataVizGA/DataVisualAnalytics_Industries-cancer/data_raw')

In [4]:
with open(raw_data_dir / 'county_cancer_stats' / 'states.txt') as f:
    states_str = f.read()
    states_list = states_str.split(', ')
    
# Convert to lower case
states_list = [str.lower(state) for state in states_list]

In [19]:
# cdq = CancerDataQuery(states_list[0])

In [20]:
# cdq.download_cancer_data('demographics')

In [8]:
#
# THE FOLLOWING IS NOW MULTITHREADED IN cancer_data_query.py
#

# for state in states_list[0:5]:
#     cdq = CancerDataQuery(state, data_dir=raw_data_dir)
#     for data_type in cdq.data_types:
#         cdq.download_cancer_data(data_type)

In [96]:
# cdq.data_keys

In [58]:
# url = "https://www.statecancerprofiles.cancer.gov/quick-profiles/index.php?statename=georgia"
# url = "https://www.statecancerprofiles.cancer.gov/quick-profiles/index.php?stateFIPS=13"

url = "https://www.statecancerprofiles.cancer.gov/quick-profiles/index.php?statename=" + states_list[0]


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
    'Content-Type': 'text/html',
}

response = requests.get(url, headers=headers)
html = response.text

In [59]:
###
# GET ALL TABLES
regex = 'class="icn icn-table" href="(.*?)"'
url_all = re.findall(regex, html)

In [60]:
###############
# GET icn-table

# Pull Unique DataSet Types
data_types = list(set([u.split('/')[1] for u in url_all]))
url_by_type = {datatype: [u for u in url_all if datatype in u] for datatype in data_types}

data_types

['demographics', 'deathrates', 'risk', 'incidencerates', 'prevalence']

In [61]:
url_by_type['demographics'][0:5]

['/demographics/index.php?stateFIPS=01&areatype=county&topic=crowd&demo=00027&race=00&sex=0&age=001&type=manyareacensus#results',
 '/demographics/index.php?stateFIPS=01&areatype=county&topic=ed&demo=00004&race=00&sex=0&age=081&type=manyareacensus#results',
 '/demographics/index.php?stateFIPS=01&areatype=county&topic=ed&demo=00005&race=00&sex=0&age=081&type=manyareacensus#results',
 '/demographics/index.php?stateFIPS=01&areatype=county&topic=ed&demo=00006&race=00&sex=0&age=081&type=manyareacensus#results',
 '/demographics/index.php?stateFIPS=01&areatype=county&topic=inc&demo=00010&race=00&sex=0&age=001&type=manyareacensus#results']

In [33]:
### OLD URL SPLITTING APPROACH

# url_deathrates = [u for u in url_all if 'deathrates' in u]
# url_risk = [u for u in url_all if 'risk' in u]
# url_prevalence = [u for u in url_all if 'prevalence' in u]
# url_demographics = [u for u in url_all if 'demographics' in u]
# url_incidencerates = [u for u in url_all if 'incidencerates' in u]

# for url in url_deathrates[:5]:
#     details = url.split('?')[1].split('&')
#     details = [d.split('#results')[0] for d in details]
#     print(details)

# for url in url_prevalence:
#     details = url.split('?')[1].split('&')
#     details = [d.split('#results')[0] for d in details]
#     print(details)

In [34]:
# Get a url

url_incidencerates[0]

NameError: name 'url_incidencerates' is not defined

In [35]:
# raw_data_dir = pathlib.Path('/Users/merrelbook/Projects/HealthDataVizGA/raw_data/CDC_CancerIncidence-GAbyCounty')

# base_url = 'https://www.statecancerprofiles.cancer.gov'

# dl_text = '&sortVariableName=rate&sortOrder=desc&output=1'
# dl_url = base_url + url_incidencerates[0].replace('#results', dl_text)
# dl_url

In [36]:

    
def download_cancer_data(data_type, url_by_type):

    # Set the proper paths
    raw_data_dir = project_data_dir / 'raw_data' / 'CDC_CancerIncidence-GAbyCounty' / data_type

    if not raw_data_dir.exists():
        raw_data_dir.mkdir(parents=True, exist_ok=True)

    # Define where to download data
    data_key = pd.DataFrame()
    base_url = 'https://www.statecancerprofiles.cancer.gov'
    
    # Define which data to donwload
    
    url_list = url_by_type[data_type]
        
    for file_idx, file_url in tqdm(enumerate(url_list), total=len(url_list)):

        if 'demographics' in file_url:
            dl_text = '&sortVariableName=value&sortOrder=asc&output=1'
        else:
            dl_text = '&sortVariableName=rate&sortOrder=desc&output=1'

        dl_url = base_url + file_url.replace('#results', dl_text)
        
        try:
            details = file_url.split('?')[1].split('&')
            details = [d.split('#results')[0] for d in details]
            details = {d.split('=')[0]: d.split('=')[1] for d in details}
        except:
            breakpoint()

        response = requests.get(dl_url, stream=True)

        new_file_name = f'{data_type}_{file_idx:03d}.csv'


        details.update({'file_name': new_file_name})
        details.update({'source_url': dl_url})

        data_key = pd.concat([data_key, pd.DataFrame(data=details, index=[file_idx])],
                             sort=True
                            )

        with open(raw_data_dir / new_file_name, "wb") as handle:
            handle.write(bytearray('------------\n', 'utf-8'))
            handle.write(bytearray('DL METADATA\n', 'utf-8'))
            handle.write(bytearray(f' > URL: {dl_url}\n', 'utf-8'))
            handle.write(bytearray(f' > Details: {details}\n', 'utf-8'))
            handle.write(bytearray('------------\n', 'utf-8'))


            for data in response.iter_content():
                handle.write(data)

    data_key.to_csv(raw_data_dir / 'DATA_KEY.csv', index=False)
    
    return data_key

In [15]:
data_types

['demographics', 'deathrates', 'risk', 'incidencerates', 'prevalence']

In [16]:
download_cancer_data('demographics', url_by_type)

100%|██████████| 32/32 [00:08<00:00,  3.86it/s]


,age,areatype,demo,file_name,race,sex,source_url,stateFIPS,topic,type
0,001,county,00027,demographics_000.csv,00,0,https://www.statecancerprofiles.cancer.gov/dem...,13,crowd,manyareacensus
1,081,county,00004,demographics_001.csv,00,0,https://www.statecancerprofiles.cancer.gov/dem...,13,ed,manyareacensus
2,081,county,00005,demographics_002.csv,00,0,https://www.statecancerprofiles.cancer.gov/dem...,13,ed,manyareacensus
3,081,county,00006,demographics_003.csv,00,0,https://www.statecancerprofiles.cancer.gov/dem...,13,ed,manyareacensus
4,001,county,00010,demographics_004.csv,00,0,https://www.statecancerprofiles.cancer.gov/dem...,13,inc,manyareacensus
5,001,county,00011,demographics_005.csv,00,0,https://www.statecancerprofiles.cancer.gov/dem...,13,inc,manyareacensus
6,006,county,00043,demographics_006.csv,00,0,https://www.statecancerprofiles.cancer.gov/dem...,13,ins,manyareacensus
7,001,county,00015,demographics_007.csv,00,0,https://www.statecancerprofiles.cancer.gov/dem...,13,lang,manyareacensus
8,001,county,00017,demographics_008.csv,00,0,https://www.statecancerprofiles.cancer.gov/dem...,13,mob,manyareacensus
9,001,county,00018,demographics_009.csv,00,0,https://www.statecancerprofiles.cancer.gov/dem...,13,mob,manyareacensus


In [14]:
# download_cancer_data('demographics', url_by_type)

In [149]:
for data_type in data_types:
    print(f'Getting data for : {data_type}')
    download_cancer_data(data_type, url_by_type)

  0%|          | 0/59 [00:00<?, ?it/s]

Getting data for : incidencerates


  0%|          | 0/39 [00:00<?, ?it/s]

Getting data for : deathrates


  2%|▏         | 1/41 [00:00<00:07,  5.23it/s]

...

 11%|█         | 1/9 [00:00<00:01,  5.98it/s]

Getting data for : prevalence


  0%|          | 0/32 [00:00<?, ?it/s]

Getting data for : demographics


100%|██████████| 32/32 [00:07<00:00,  4.21it/s]


In [37]:
# d.split['='][0]: d.split['='][1]

---
## Cleaning per data type

In [17]:
def make_meta_df(idx_max, subset_key):
    for idx in range(idx_max):
        df = pd.DataFrame(subset_key).transpose()
        df.index = [idx]
        yield df

def skip_to(fle, line,**kwargs):
    '''Source: https://stackoverflow.com/questions/34028511/skipping-unknown-number-of-lines-to-read-the-header-python-pandas'''
    if os.stat(fle).st_size == 0:
        raise ValueError("File is empty")
    with open(fle, errors='replace') as f:
        pos = 0
        cur_line = f.readline()
        while not cur_line.startswith(line):
            pos = f.tell()
            cur_line = f.readline()
        f.seek(pos)
        return pd.read_csv(f, **kwargs)
        
def gen_subsets(df_key, raw_data_dir, cols_to_clean=None):
    for idx, subset_key in tqdm(df_key.iterrows(), total=len(df_key)):

        # df_subset = pd.read_csv(raw_data_dir / subset_key.file_name, header=9, encoding = "ISO-8859-1").dropna()
        df_subset = skip_to(raw_data_dir / subset_key.file_name, 'County')

        df_subset.columns = [c.strip().lower() for c in df_subset.columns]
        df_subset = df_subset.dropna(subset=['fips'])
        
        # Join with metadata
        df_subset = df_subset.join(
            pd.concat(make_meta_df(len(df_subset), subset_key))
        )
        
        # Rename the columns
        if cols_to_clean is not None:
            try:
                df_subset.columns = cols_to_clean
            except ValueError:
                print(subset_key.file_name)
                print('---')
                print(df_subset.columns)
                print('---')
  
        yield df_subset

---
### Process Death Rate Data

Processing Steps
1. Load Each Pre-downloaded data
2. Clean (rename) columns
3. Join metadata to each row of subset
4. Combine datasets
5. Drow all rows with NaN
6. Change areatype for state and country entries

In [18]:
data_type = 'deathrates'

# Set the data path
project_data_dir = pathlib.Path('/Users/merrelbook/Projects/HealthDataVizGA')
raw_data_dir = project_data_dir / 'raw_data' / 'CDC_CancerIncidence-GAbyCounty' / data_type

# Set the data path
project_data_dir = pathlib.Path('/Users/merrelbook/Projects/HealthDataVizGA')
raw_data_dir = project_data_dir / 'raw_data' / 'CDC_CancerIncidence-GAbyCounty' / 'deathrates'

# Load the data key
df_key = pd.read_csv(raw_data_dir / 'DATA_KEY.csv')

# Load and clean/rename columns
cols_to_clean= ['locale', 'fips', 'met_health_obj', 
            'deathrate_per_100000', 'deathrate_lower_95_confidence', 'deathrate_upper_95_confidence',
            'annual_count_avg', 'recent_trend_str',
            'trend_last_5', 'trend_last_5_lower_95_confidence', 'trend_last_5_upper_95_confidence',
            'stateFIPS', 'areatype', 'cancer', 'race', 'sex', 'age', 'type','file_name', 'source_url']

df = pd.concat(
    gen_subsets(df_key, raw_data_dir, cols_to_clean)
)

# Change areatype for state and country entries
df.loc[df.locale == 'Georgia', 'areatype'] = "state"
df.loc[df.locale == 'United States', 'areatype'] = "country"

df.to_csv(project_data_dir / 'raw_data' / 'CDC_CancerIncidence-GAbyCounty' / data_type / 'data_join_clean.csv',
          index=False
         )
print(f'Data Joined and cleaned for data type:\n -> {data_type}')
df.head()

100%|██████████| 39/39 [00:09<00:00,  3.97it/s]

Data Joined and cleaned for data type:
 -> deathrates


,locale,fips,met_health_obj,deathrate_per_100000,deathrate_lower_95_confidence,deathrate_upper_95_confidence,annual_count_avg,recent_trend_str,trend_last_5,trend_last_5_lower_95_confidence,trend_last_5_upper_95_confidence,stateFIPS,areatype,cancer,race,sex,age,type,file_name,source_url
0,Georgia,13000.0,No,165.6,164.4,166.7,16650,falling,-1.6,-1.7,-1.4,1,state,1,deathrates_000.csv,0,0,https://www.statecancerprofiles.cancer.gov/dea...,13,death
1,United States,0.0,Yes,161,160.8,161.1,590623,falling,-1.5,-1.6,-1.5,1,country,1,deathrates_000.csv,0,0,https://www.statecancerprofiles.cancer.gov/dea...,13,death
2,Turner County,13287.0,No,236.4,196.3,282.9,25,stable,0.5,-0.5,1.5,1,county,1,deathrates_000.csv,0,0,https://www.statecancerprofiles.cancer.gov/dea...,13,death
3,Glascock County,13125.0,No,235.3,168.9,320.6,9,stable,1.1,-1.0,3.3,1,county,1,deathrates_000.csv,0,0,https://www.statecancerprofiles.cancer.gov/dea...,13,death
4,Butts County,13035.0,No,230.7,205,258.9,61,stable,0.1,-0.8,0.9,1,county,1,deathrates_000.csv,0,0,https://www.statecancerprofiles.cancer.gov/dea...,13,death


---
### Process Incidence Data

Processing Steps
1. Load Each Pre-downloaded data
2. Drop Duplicates from DATA KEY
3. Split data key to process ALL STAGE and LATE STAGE data sets separately (different column names)
4. Drow all rows with NaN
5. Clean (rename) columns
6. Join metadata to each row of subset
7. Combine data subsets
8. Re-join ALL STAGE and LATE STAGE data sets

In [227]:
#
# PROCESS INCIDENCE RATE DATA
#

data_type = 'incidencerates'

# Set the data path
project_data_dir = pathlib.Path('/Users/merrelbook/Projects/HealthDataVizGA')
raw_data_dir = project_data_dir / 'raw_data' / 'CDC_CancerIncidence-GAbyCounty' / data_type

# Load the data key
df_key = pd.read_csv(raw_data_dir / 'DATA_KEY.csv')

# Drop all the duplicates from data key
df_key = df_key.drop_duplicates(
    subset=[c for c in df_key.columns if 'file_name' not in c]
    # Ignore the file_name column for deduplication
)

#
# ALL STAGE DATA  -  Must split 'all stage' and 'late stage' data sets due to difference in columns
#

# Split out the data by stage keys
df_key_by_stage = df_key[~df_key.stage.isna()]
df_key = df_key[df_key.stage.isna()]

# Load and clean/rename columns
cols_to_clean= ['locale', 'fips', 'met_health_obj', 
            'incidence rate_per_100000', 'incidence rate_lower_95_confidence', 'incidence rate_upper_95_confidence',
            'annual_count_avg', 'recent_trend_str',
            'trend_last_5', 'trend_last_5_lower_95_confidence', 'trend_last_5_upper_95_confidence',
            'age', 'areatype', 'cancer', 'file_name', 'race', 'sex', 'source_url', 
            'stage', 'stateFIPS', 'type']

print('Processing data for all cancer stages')
df_all_stage = pd.concat(
    gen_subsets(df_key, raw_data_dir, cols_to_clean=cols_to_clean)
)

#
# LATE STAGE DATA
#

# Load and clean/rename columns
cols_to_clean= ['locale', 'fips', 'met_health_obj', 
            'incidence rate_per_100000', 'incidence rate_lower_95_confidence', 'incidence rate_upper_95_confidence',
            'annual_count_avg', 'late_stage_%',
            'age', 'areatype', 'cancer', 'file_name', 'race', 'sex', 'source_url', 
            'stage', 'stateFIPS', 'type']

print('Processing data for LATE cancer stages')
df_late_stage = pd.concat(
    gen_subsets(df_key_by_stage, raw_data_dir, cols_to_clean=cols_to_clean),
    sort=False
)

#
# MERGE datasets from all-stage and late-stage
#

df = pd.concat([df_all_stage, df_late_stage], sort=False)

# Change areatype for state and country entries
df.loc[df.locale == 'Georgia(7,8)', 'areatype'] = "state"
df.loc[df.locale == 'US (SEER+NPCR)(1,10)', 'areatype'] = "country"

# Save out the cleaned and joined dataset
df.to_csv(project_data_dir / 'raw_data' / 'CDC_CancerIncidence-GAbyCounty' / data_type / 'data_join_clean.csv',
          index=False
         )

  0%|          | 0/36 [00:00<?, ?it/s]

Processing data for all cancer stages


  0%|          | 0/19 [00:00<?, ?it/s]

Processing data for LATE cancer stages


100%|██████████| 19/19 [00:04<00:00,  4.02it/s]


In [228]:
df.head()

,locale,fips,met_health_obj,incidence rate_per_100000,incidence rate_lower_95_confidence,incidence rate_upper_95_confidence,annual_count_avg,recent_trend_str,trend_last_5,trend_last_5_lower_95_confidence,...,areatype,cancer,file_name,race,sex,source_url,stage,stateFIPS,type,late_stage_%
0,"Georgia(7,8)",13000.0,***,466.4,464.5,468.3,49003,falling,-0.3,-0.5,...,state,1,incidencerates_000.csv,0,0,https://www.statecancerprofiles.cancer.gov/inc...,NaN,13,incd,NaN
1,"US (SEER+NPCR)(1,10)",0.0,***,448,447.7,448.4,1.63811e+06,falling,-0.9,-1.6,...,country,1,incidencerates_000.csv,0,0,https://www.statecancerprofiles.cancer.gov/inc...,NaN,13,incd,NaN
2,"Pickens County(7,8)",13227.0,***,580.8,547.2,616.1,249,rising,1.2,0.2,...,county,1,incidencerates_000.csv,0,0,https://www.statecancerprofiles.cancer.gov/inc...,NaN,13,incd,NaN
3,"Towns County(7,8)",13281.0,***,562.7,509.9,620.5,116,stable,0.1,-1.1,...,county,1,incidencerates_000.csv,0,0,https://www.statecancerprofiles.cancer.gov/inc...,NaN,13,incd,NaN
4,"Union County(7,8)",13291.0,***,546,510.5,583.9,226,stable,0.4,-0.7,...,county,1,incidencerates_000.csv,0,0,https://www.statecancerprofiles.cancer.gov/inc...,NaN,13,incd,NaN


In [108]:
# ###################################
# Queries to Split incidence datasets

# # Split out the data by stage keys
# df[~df.stage.isna()]

# # Split out the data by age keys
# criterion = ((df.age!=1) & (df.cancer==1))

# # Split out the data by sex keys
# criterion = ((df.sex!=0) & (df.cancer==1))

# # Split out the data by race keys
# criterion = ((df.race!=0))

# # Split out the data by cancer keys
# criterion = ((df.cancer!=1))

# # All cancer, race, sex, age, stage
# df_key_by_all = df_key

In [109]:
data_types

['incidencerates', 'deathrates', 'risk', 'prevalence', 'demographics']

---
### Process Demographic Data

Processing Steps
1. Load Each Pre-downloaded data
2. Drop Duplicates from DATA KEY

In [52]:
#
# PROCESS Risk DATA
#

data_type = 'demographics'

# Set the data path
project_data_dir = pathlib.Path('/Users/merrelbook/Projects/HealthDataVizGA')
raw_data_dir = project_data_dir / 'raw_data' / 'CDC_CancerIncidence-GAbyCounty' / data_type

# Load the data key
df_key = pd.read_csv(raw_data_dir / 'DATA_KEY.csv')

# Drop all the duplicates from data key
df_key = df_key.drop_duplicates(
    subset=[c for c in df_key.columns if 'file_name' not in c]
    # Ignore the file_name column for deduplication
)

In [53]:
df_key.head(1)

,age,areatype,demo,file_name,race,sex,source_url,stateFIPS,topic,type
0,1,county,27,demographics_000.csv,0,0,https://www.statecancerprofiles.cancer.gov/dem...,13,crowd,manyareacensus


In [8]:
df_demographics.columns

Index(['county', 'fips', 'value (percent)',
       'households (with >1 person per room)', 'rank within us', 'age',
       'areatype', 'demo', 'file_name', 'race', 'sex', 'source_url',
       'stateFIPS', 'topic', 'type', 'people (education: less than 9th grade)',
       'people(education: less than high school)',
       'people (education: at least bachelors degree)', 'value (dollars)',
       'people (uninsured)', 'households (language isolation)',
       'people (haven't moved)', 'people (moved within county)',
       'people (moved from different county in same state)',
       'people (moved from different state)', 'people (moved from outside us)',
       'people (age under 18)', 'people (age 18-39)', 'people (age 40-64)',
       'people (age 40 and over)', 'people (age 50 and over)',
       'people (age 65 and over)', 'people (ai/an)', 'people (api)',
       'people (black)', 'people (foreign born)', 'people (hispanic)',
       'people (non-hispanic [origin recode])', 'people (whi

In [78]:
print('Processing data for all cancer stages')


# df_demographics = pd.concat(
#     gen_subsets(df_key, raw_data_dir, cols_to_clean=None),
#     sort=False
# ).reset_index(drop=True)

df_demographics.rename(columns={'county': 'locale'}, inplace=True)

# # Change areatype for state and country entries
df_demographics.loc[df_demographics.locale == 'Georgia', 'areatype'] = "state"
df_demographics.loc[df_demographics.locale == 'United States', 'areatype'] = "country"

df_demographics.to_csv(project_data_dir / 'raw_data' / 'CDC_CancerIncidence-GAbyCounty' / data_type / 'data_join_clean.csv',
          index=False
         )
print(f'Data Joined and cleaned for data type:\n -> {data_type}')
df_demographics.head()

Processing data for all cancer stages


In [41]:
# Subdataset 1 - Demographics: Crowding
# - Also include 'value (percent)', 'rank within us'
criterion = ~df_demographics['households (with >1 person per room)'].isna()

# # Subdataset 2 - Demographics: Education
# - Also include 'value (percent)', 'rank within us'
criterion = ( (~ df_demographics['people (education: less than 9th grade)'].isna()) \
            | (~ df_demographics['people(education: less than high school)'].isna()) \
            | (~ df_demographics['people (education: at least bachelors degree)'].isna()) )

# Subdataset 3 - Demographics: Income
# - Split Further by 
#   - demo=10 --> Median family income, 2013-2017
#   - demo=11 --> Median household income, 2013-2017
criterion = (~df_demographics["value (dollars)"].isna())

# Subdataset 4 - Demographics: Insurance
# - Also include 'value (percent)', 'rank within us'
# - Percent uninsured in demographic group, people at or below 138% of poverty, 2017, Ages <65
criterion = (~df_demographics['people (uninsured)'].isna())

# Subdataset 5 - Demographics: Language
# - Also include 'value (percent)', 'rank within us'
# - Language isolation, 2013-2017:  https://www.statecancerprofiles.cancer.gov/dictionary.php#non-english
criterion = (~df_demographics['households (language isolation)'].isna())

# Subdataset 6 - Demographics: Mobility
# - Also include 'value (percent)', 'rank within us'
criterion = ( (~ df_demographics["people (haven't moved)"].isna()) \
            | (~ df_demographics["people (moved within county)"].isna()) \
            | (~ df_demographics["people (moved from different county in same state)"].isna()) \
            | (~ df_demographics["people (moved from different state)"].isna()) \
            | (~ df_demographics["people (moved from outside us)"].isna()) )


# Subdataset 7 - Demographics: Population - Ages
# - Also include 'value (percent)', 'rank within us'
criterion = ( (~ df_demographics["people (age under 18)"].isna()) \
            | (~ df_demographics["people (age 18-39)"].isna()) \
            | (~ df_demographics["people (age 40-64)"].isna()) \
            | (~ df_demographics["people (age 40 and over)"].isna()) \
            | (~ df_demographics["people (age 50 and over)"].isna()) \
            | (~ df_demographics["people (age 65 and over)"].isna()) )


# Subdataset 8 - Demographics: Population -  Race/Ethnicity
# - Also include 'value (percent)', 'rank within us'
criterion = ( (~ df_demographics["people (ai/an)"].isna()) \
            | (~ df_demographics["people (api)"].isna()) \
            | (~ df_demographics["people (black)"].isna()) \
            | (~ df_demographics["people (foreign born)"].isna()) \
            | (~ df_demographics["people (hispanic)"].isna()) \
            | (~ df_demographics["people (non-hispanic [origin recode])"].isna()) \
            | (~ df_demographics["people (white)"].isna()) )


# Subdataset 9 - Demographics: Population -  Sex
# - Also include 'value (percent)', 'rank within us'
criterion = ( (~ df_demographics['people (male)'].isna()) \
            | (~ df_demographics['people (female)'].isna()) )


# Subdataset 9 - Demographics: Poverty
# - Also include 'value (percent)', 'rank within us'
criterion = ( (~ df_demographics['families (below poverty)'].isna()) \
            | (~ df_demographics['people (below poverty)'].isna()) \
            | (~ df_demographics['people (<150% of poverty)'].isna()) )

# Subdataset 5 - Demographics: Workforce/Unemployment
# - Also include 'value (percent)', 'rank within us'
criterion = (~df_demographics['people (unemployed)'].isna())